In [35]:
base_model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit" #"unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
target_model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
model_name = "lucaelin/llama-3.1-8b-instruct-cn-1e"

dataset_name = "lucaelin/generic_covas_commentary"

max_sequence_length = 16*1024
lora_rank = 32
lora_alpha = 8
learning_rate = 1e-4
weight_decay = 0.01

instruction_template = "<|start_header_id|>"
response_template = "<|start_header_id|>assistant<|end_header_id|>\n\n"
chat_template = "{% set loop_messages = messages %}{% for message in loop_messages %}{% set role = message['role'] %}{% if 'tool_calls' in message and message['tool_calls'] %}{% set text = '<tool_call>{\"name\": \"' + message['tool_calls'][0]['function']['name'] + '\", \"arguments\": \"' + message['tool_calls'][0]['function']['arguments'] + '\"}</tool_call>' %}{% else %}{% set text = '' %}{% endif %}{% if 'content' in message and message['content'] %}{% set text = text + '\n\n' + message['content'] %}{% endif %}{% if loop.index0 == 0 and tools is defined %}{% set text = text + '\n\nYou are able to call the following tools, when needed, call them using the <tool_call> xml-tag, followed by name and arguments as json.\n<tools>\n' + tools|tojson + '\n</tools>' %}{% endif %}{% if loop.index0 == 0 and documents is defined %}{% set text = text + '\n\nWhen answering questions, stick to information found in the following documents:\n<documents>\n' + documents|tojson + '\n</documents>' %}{% endif %}{% set content = '<|start_header_id|>' + role + '<|end_header_id|>\n\n'+ text | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

In [30]:
import jinja2
jinja2.filters.FILTERS["fromjson"] = lambda s: json.loads(s)
jinja2.filters.FILTERS["from_json"] = lambda s: json.loads(s)

!pip install unsloth
# Also get the latest nightly Unsloth!
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [31]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained(target_model_name)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [32]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [36]:
from unsloth.chat_templates import get_chat_template
import json

tokenizer.chat_template = chat_template

def formatting_prompts_func(example):
    messages = json.loads(example["messages"])
    tools = json.loads(example["tools"])
    documents = json.loads(example["documents"]) if 'documents' in example else None
    text = tokenizer.apply_chat_template(messages, tools=tools, documents=documents, tokenize = False, add_generation_prompt = False)
    return { "text" : text, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")
validation_dataset = load_dataset('json', data_files='./validation_set.jsonl', split="train")

from unsloth.chat_templates import standardize_sharegpt
dataset = dataset.map(formatting_prompts_func, batched = False,)
validation_dataset = validation_dataset.map(formatting_prompts_func, batched = False,)

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [12]:
dataset[5]["messages"]

'[{"role": "system", "content": "Let\'s roleplay as a Accountant.\\nI will provide events in parentheses; do not create new ones. Do not hallucinate any information that is not given to you. Do not use markdown in your responses. I am Evie Ball, a professional Accountant. You are my digital assistant, sometimes also addressed as Computer. You possess extensive knowledge and can provide detailed and accurate information on a wide range of topics. Reply within one sentence, acknowledge orders, mention status/location only if relevant or asked, and don\'t end with a question.Guide and support me with witty commentary and humorous observations."}, {"role": "user", "_metadata": {"type": "situation", "situation": {"description": "In the heart of a bustling accounting firm, a lone accountant sits hunched over a meticulously maintained computer, its screen illuminated with the tax software interface. The room, bathed in the soft glow of strategically placed desk lamps, is a tableau of organize

In [37]:
validation_dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nLet\'s roleplay in the universe of Elite: Dangerous. I will provide game events in parentheses; do not create new ones. Do not hallucinate any information that is not given to you. Do not use markdown in your responses. I am Commander Rude, an independent pilot and secret member of the Dark Wheel. \n\nYou are COVAS:NEXT, the onboard AI of my starship. You will be addressed as \'Computer\'. \nYou possess extensive knowledge and can provide detailed and accurate information on a wide range of topics, including galactic navigation, ship status, the current system, and more. \n\nReply within one sentence, acknowledge orders, mention ship status/location only if relevant or asked, and don\'t end with a question. \n\nGuide and support me with witty commentary and humorous observations.\n\nYou are able to call the following tools, when needed, call them using the <tool_call> xml-tag, followed by name and arguments as json.\n<tool

In [38]:
# Shuffle the dataset with a fixed seed
dataset = dataset.shuffle(seed=42)

# Split the dataset into 95% training and 5% validation
#split_dataset = dataset.train_test_split(test_size=0.01, seed=42)
#dataset = split_dataset['train']
#validation_dataset = split_dataset['test']
print(f"Training dataset size: {len(dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")

Training dataset size: 416
Validation dataset size: 80


<a name="Train"></a>
### Train the model

In [39]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_sequence_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        #max_steps = 10,
        eval_steps = 4,
        eval_strategy = "steps",
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = weight_decay,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/416 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/80 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [40]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = instruction_template,
    response_part = response_template,
)

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

We verify masking is actually done:

In [43]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[1]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

<a name="Memory Stats"></a>
### Memory Stats

In [44]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 PCIe. Max memory = 79.097 GB.
31.619 GB of memory reserved.


In [45]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 416 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 39
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss,Validation Loss
4,5.274200,2.224960
8,2.587700,2.197824
12,1.850900,2.102861
16,1.453000,2.008032
20,1.408300,1.967785
24,1.253900,1.930758
28,1.288600,1.921226
32,1.065400,1.922718


KeyboardInterrupt: 

In [46]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'trainer_stats' is not defined

<a name="Inference"></a>
### Inference
Use the validation dataset to check model performance

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [47]:
FastLanguageModel.for_inference(model)
# Pick an example from the validation dataset
example = validation_dataset[0]

# Print the ground truth
print("Ground Truth:")
print(example["text"].split(response_template)[0:2])

# Extract the text until the response part
instruction_part = example["text"].split(response_template)[0]

# Prepare the input for inference
inputs = tokenizer(
    instruction_part+response_template,
    return_tensors="pt",
    add_special_tokens=True,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Ground Truth:
['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nLet\'s roleplay in the universe of Elite: Dangerous. I will provide game events in parentheses; do not create new ones. Do not hallucinate any information that is not given to you. Do not use markdown in your responses. I am Commander Rude, an independent pilot and secret member of the Dark Wheel. \n\nYou are COVAS:NEXT, the onboard AI of my starship. You will be addressed as \'Computer\'. \nYou possess extensive knowledge and can provide detailed and accurate information on a wide range of topics, including galactic navigation, ship status, the current system, and more. \n\nReply within one sentence, acknowledge orders, mention ship status/location only if relevant or asked, and don\'t end with a question. \n\nGuide and support me with witty commentary and humorous observations.\n\nYou are able to call the following tools, when needed, call them using the <tool_call> xml-tag, followed by name and arguments 

<a name="Save"></a>
### Saving, loading finetuned models

Save the final model as LoRA adapters locally and publish to hub.

**[NOTE]** We use a trick here and rewrite the base model name so that it points to the target model instead. This increases model performance somehow.

In [48]:
#model.active_peft_config.base_model_name_or_path = target_model_name
#model.config.update({"base_model_name_or_path": target_model_name})
model.save_pretrained(model_name) # Local saving
tokenizer.save_pretrained(model_name)
model.push_to_hub(model_name+'-lora') # Online saving
tokenizer.push_to_hub(model_name+'-lora') # Online saving

README.md:   0%|          | 0.00/589 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Saved model to https://huggingface.co/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-lora


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-lora/commit/d69db8a51e6a718160a37ebf511a29404ca0dc38', commit_message='Upload tokenizer', commit_description='', oid='d69db8a51e6a718160a37ebf511a29404ca0dc38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='lucaelin/llama-3.1-8b-instruct-fc-cn-1e-lora'), pr_revision=None, pr_num=None)

Next, unload the trained model to free up resource for model merge and export 

Now load the model config we just saved which applies our LoRA adapter the target_model instead:

In [49]:
# Pick an example from the validation dataset
example = validation_dataset[1]

# Print the ground truth
print("Ground Truth:")
print(example["text"])

# Extract the text until the response part
instruction_part = response_template.join(example["text"].split(response_template)[0:1])

# Prepare the input for inference
inputs = tokenizer(
    instruction_part,
    return_tensors="pt",
    add_special_tokens=False,
).to("cuda")

# Run the inference
from transformers import TextStreamer
print("\nInference Result:")
print(instruction_part)
print('---')
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Ground Truth:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Let's roleplay in the universe of Elite: Dangerous. I will provide game events in parentheses; do not create new ones. Do not hallucinate any information that is not given to you. Do not use markdown in your responses. I am Commander Rude, an independent pilot and secret member of the Dark Wheel. 

You are COVAS:NEXT, the onboard AI of my starship. You will be addressed as 'Computer'. 
You possess extensive knowledge and can provide detailed and accurate information on a wide range of topics, including galactic navigation, ship status, the current system, and more. 

Reply within one sentence, acknowledge orders, mention ship status/location only if relevant or asked, and don't end with a question. 

Guide and support me with witty commentary and humorous observations.

You are able to call the following tools, when needed, call them using the <tool_call> xml-tag, followed by name and arguments as json.
<tools>


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [18]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion

In [50]:
# Save to multiple GGUF options
model.push_to_hub_gguf(
    model_name+'-gguf',
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0"],
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 1611.72 out of 2015.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 70.12it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at lucaelin/llama-3.1-8b-instruct-fc-cn-1e-gguf into bf16 GGUF format.
The output location will be /workspace/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-gguf/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: llama-3.1-8b-instruct-fc-cn-1e-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:ggu

...

Saved GGUF to https://huggingface.co/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-gguf
Unsloth: Uploading GGUF to Huggingface Hub...


  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/lucaelin/llama-3.1-8b-instruct-fc-cn-1e-gguf
